In [16]:
import pandas as pd
from pathlib import Path
from typing import Dict

In [17]:
county_dfs: Dict[str, pd.DataFrame] = {}

for path in Path("Resources/Budget").glob("Budget_by_County_*.csv"):
    name_parts = path.stem.split("_")
    year = name_parts[-1]
    if year == "Merged":
        continue
    df = pd.read_csv(path)
    df["Year"] = int(year)
    county_dfs[year] = df

df = pd.DataFrame(columns=county_dfs["2010"].columns)

for year, year_df in county_dfs.items():
    for col in [col for col in year_df.columns if col not in ["State","County/ Congressional District"]]:
        year_df[col] = pd.to_numeric(year_df[col], errors="coerce")
        year_df = year_df.fillna(0)
        year_df[col] = year_df[col].astype("int")
    df = df.merge(year_df, how="outer")
merged_county_df = df

merged_county_df.to_csv(f"Resources/Budget/Budget_by_County_Merged.csv", index=False)

In [18]:
state_dfs: Dict[str, pd.DataFrame] = {}

for path in Path("Resources/Budget").glob("State_Total_*.csv"):
    name_parts = path.stem.split("_")
    year = name_parts[-1]
    if year == "Merged":
        continue
    df = pd.read_csv(path)
    df["Year"] = int(year)
    for row in range(len(df.index)):
        df.iloc[row, 0] = df.iloc[row, 0].strip(" ")
    state_dfs[year] = df

df = pd.DataFrame(columns=state_dfs["2010"].columns)

for year, year_df in state_dfs.items():
    for col in [col for col in year_df.columns if col != "State"]:
        year_df[col] = pd.to_numeric(year_df[col], errors="coerce")
        year_df = year_df.fillna(0)
        year_df[col] = year_df[col].astype("int")
    df = df.merge(year_df, how="outer")

df.to_csv(f"Resources/Budget/State_Total_Merged.csv", index=False)